In [ ]:
from PIL import Image
import numpy as np

# Function to load an image and convert it to RGB format
def load_image(image_path):
    image = Image.open(image_path)
    return np.array(image)

# Function to save the modified image with hidden data
def save_image(image_array, output_path):
    image = Image.fromarray(image_array)
    image.save(output_path)

# Convert data to binary string
def data_to_bin(data):
    binary_data = ''.join(format(byte, '08b') for byte in data)
    return binary_data

# Function to hide the binary data in the image (using LSB)
def hide_data_in_image(image_array, binary_data):
    data_len = len(binary_data)
    data_idx = 0
    
    # Iterate through each pixel and each color channel
    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            for k in range(3):  # RGB channels
                if data_idx < data_len:
                    pixel_value = image_array[i, j, k]
                    # Set the LSB to the binary data
                    image_array[i, j, k] = (pixel_value & 0xFE) | int(binary_data[data_idx])
                    data_idx += 1
    return image_array

# Function to encode data into an image
def encode_data(image_path, secret_data, output_image_path):
    image_array = load_image(image_path)
    binary_data = data_to_bin(secret_data.encode('utf-8'))  # Convert secret data to binary
    modified_image_array = hide_data_in_image(image_array, binary_data)
    save_image(modified_image_array, output_image_path)
    print(f"Data hidden successfully! Image saved at {output_image_path}")

# Example usage
secret_text = "This is a secret message"
input_image_path = "input_image.png"
output_image_path = "output_image.png"
encode_data(input_image_path, secret_text, output_image_path)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras import regularizers

# Define the autoencoder architecture
def build_autoencoder(input_shape):
    input_img = Input(shape=input_shape)
    x = Flatten()(input_img)
    x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    encoded = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    
    # Decoder part of the autoencoder
    x = Dense(256, activation='relu')(encoded)
    x = Dense(512, activation='relu')(x)
    decoded = Dense(np.prod(input_shape), activation='sigmoid')(x)
    decoded = Reshape(input_shape)(decoded)
    
    # Model
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    
    return autoencoder

# Train the autoencoder
def train_autoencoder(image_array):
    image_array = image_array.astype('float32') / 255.0  # Normalize the image
    autoencoder = build_autoencoder(image_array.shape)
    
    # Train on the image data (here, using a single image for demonstration)
    autoencoder.fit(image_array, image_array, epochs=50, batch_size=1, shuffle=True)
    return autoencoder

# Example usage: Train the autoencoder with the image data
input_image_array = load_image(input_image_path)
trained_model = train_autoencoder(input_image_array)

In [ ]:
# Function to decode the binary data from the image
def decode_data_from_image(image_array):
    binary_data = ""
    
    # Extract LSBs from the image
    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            for k in range(3):  # RGB channels
                binary_data += str(image_array[i, j, k] & 1)
    
    # Convert binary data back to text
    binary_data = binary_data[:len(binary_data) - len(binary_data) % 8]  # Remove incomplete byte
    secret_data = bytearray()
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        secret_data.append(int(byte, 2))
    
    return secret_data.decode('utf-8')

# Example usage: Decode the hidden data from the modified image
modified_image = load_image("output_image.png")
decoded_message = decode_data_from_image(modified_image)
print(f"Decoded message: {decoded_message}")